In [1]:
import os
import sys

from cltk.corpus.greek.tlg.parse_tlg_indices import get_id_author
from cltk.corpus.utils.formatter import assemble_tlg_author_filepaths
from nltk.tokenize.punkt import PunktLanguageVars

In [2]:
p = PunktLanguageVars()

# Lexical diveristy by author

## Original text with diacritics

In [3]:
cleaned_dir = os.path.expanduser('~/cltk_data/greek/text/tlg/plaintext_clean')
dir_contents = os.listdir(cleaned_dir)

all_tokens_list = []

for doc_count, file in enumerate(dir_contents):
    file_path = os.path.join(cleaned_dir, file)
    with open(file_path) as fo:
        text = fo.read().lower()
    text = ''.join([char for char in text if char not in ['.']])
    tokens = p.word_tokenize(text)
    all_tokens_list += tokens

print('Total author files:', doc_count)
print('Total words:', len(all_tokens_list))
all_tokens_unique = set(all_tokens_list)
print('Total unique words:', len(all_tokens_unique))

Total author files: 1822
Total words: 72057716
Total unique words: 1515193


In [4]:
map_id_author = get_id_author()

In [5]:
# Words and unique words per author
map_id_word_counts = {}
for file in dir_contents:
    map_word_counts = {}
    file_path = os.path.join(cleaned_dir, file)
    author_id = file[3:-4]
    author = map_id_author[author_id]
    with open(file_path) as fo:
        text = fo.read().lower()
    text = ''.join([char for char in text if char not in ['.']])
    tokens = p.word_tokenize(text)
    map_word_counts['name'] = author
    map_word_counts['word_count_all'] = len(tokens)
    map_word_counts['word_count_unique'] = len(set(tokens))
    map_id_word_counts[author_id] = map_word_counts
    print(author)
    print('    ', 'Total words', len(tokens))
    print('    ', 'Total unique words', len(set(tokens)))

Lepidus Hist.
     Total words 122
     Total unique words 98
Melito Trag.
     Total words 0
     Total unique words 0
[Polyidus] Trag.
     Total words 19
     Total unique words 18
Archippus Comic.
     Total words 613
     Total unique words 304
Martyrium Potamiaenae Et Basilidis
     Total words 381
     Total unique words 254
Acta Phileae
     Total words 881
     Total unique words 443
Menecrates Hist.
     Total words 200
     Total unique words 143
Marinus Phil.
     Total words 2563
     Total unique words 880
Troilus Soph.
     Total words 3224
     Total unique words 1259
Apollinaris Theol.
     Total words 53263
     Total unique words 12353
Antileon Hist.
     Total words 30
     Total unique words 25
Hermas Scr. Eccl., Pastor Hermae
     Total words 27950
     Total unique words 5486
Agatharchides Geogr.
     Total words 18653
     Total unique words 6515
Mnaseas Perieg.
     Total words 2196
     Total unique words 1130
Pausaniae I Et Xerxis Epistulae
     Total words 1

## Original text with diacritics

# Lexical diversity by genre